# 🤖 GetAround - Machine Learning : Prédiction de Prix

## Objectif
Créer un modèle de Machine Learning pour prédire les prix optimaux de location de voitures.

## Étapes
1. Exploration des données (EDA)
2. Préparation des données
3. Feature Engineering
4. Entraînement de plusieurs modèles
5. Évaluation et sélection du meilleur modèle
6. Optimisation des hyperparamètres
7. Sauvegarde du modèle pour l'API

## 1. Chargement des bibliothèques

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

# Model persistence
import joblib

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)

# Random state pour la reproductibilité
RANDOM_STATE = 42

print("✅ Bibliothèques chargées avec succès")

✅ Bibliothèques chargées avec succès


## 2. Chargement des données

In [2]:
# Charger le fichier CSV
df_pricing = pd.read_csv('../data/get_around_pricing_project.csv')

print(f"📊 Données chargées : {len(df_pricing):,} lignes et {len(df_pricing.columns)} colonnes")
print(f"\n📦 Taille mémoire : {df_pricing.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

📊 Données chargées : 4,843 lignes et 15 colonnes

📦 Taille mémoire : 1.20 MB


## 3. Exploration initiale des données

In [3]:
# Aperçu des données
print("="*80)
print("APERÇU DES DONNÉES")
print("="*80)
display(df_pricing.head(10))

APERÇU DES DONNÉES


,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183
5,5,Citroën,152352,225,petrol,black,convertible,True,True,False,False,True,True,True,131
6,6,Citroën,205219,145,diesel,grey,convertible,True,True,False,False,True,True,True,111
7,7,Citroën,115560,105,petrol,white,convertible,True,True,False,False,False,True,True,78
8,8,Peugeot,123886,125,petrol,black,convertible,True,False,False,False,False,True,True,79
9,9,Citroën,139541,135,diesel,white,convertible,False,False,False,False,True,False,True,132


In [4]:
# Informations sur les colonnes
print("="*80)
print("INFORMATIONS SUR LES COLONNES")
print("="*80)
df_pricing.info()

INFORMATIONS SUR LES COLONNES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 4843 non-null   int64 
 1   model_key                  4843 non-null   object
 2   mileage                    4843 non-null   int64 
 3   engine_power               4843 non-null   int64 
 4   fuel                       4843 non-null   object
 5   paint_color                4843 non-null   object
 6   car_type                   4843 non-null   object
 7   private_parking_available  4843 non-null   bool  
 8   has_gps                    4843 non-null   bool  
 9   has_air_conditioning       4843 non-null   bool  
 10  automatic_car              4843 non-null   bool  
 11  has_getaround_connect      4843 non-null   bool  
 12  has_speed_regulator        4843 non-null   bool  
 13  winter_tires               4843 n

In [5]:
# Statistiques descriptives
print("="*80)
print("STATISTIQUES DESCRIPTIVES")
print("="*80)
display(df_pricing.describe())

STATISTIQUES DESCRIPTIVES


,Unnamed: 0,mileage,engine_power,rental_price_per_day
count,4843.000000,4.843000e+03,4843.00000,4843.000000
mean,2421.000000,1.409628e+05,128.98823,121.214536
std,1398.198007,6.019674e+04,38.99336,33.568268
min,0.000000,-6.400000e+01,0.00000,10.000000
25%,1210.500000,1.029135e+05,100.00000,104.000000
50%,2421.000000,1.410800e+05,120.00000,119.000000
75%,3631.500000,1.751955e+05,135.00000,136.000000
max,4842.000000,1.000376e+06,423.00000,422.000000


In [6]:
# Valeurs manquantes
print("="*80)
print("VALEURS MANQUANTES")
print("="*80)
missing = df_pricing.isnull().sum()
missing_pct = (missing / len(df_pricing) * 100).round(2)
missing_df = pd.DataFrame({
    'Nombre': missing,
    'Pourcentage': missing_pct
})
display(missing_df[missing_df['Nombre'] > 0].sort_values('Nombre', ascending=False))

if missing.sum() == 0:
    print("\n✅ Aucune valeur manquante !")

VALEURS MANQUANTES


,Nombre,Pourcentage



✅ Aucune valeur manquante !


In [7]:
# Types de données
print("="*80)
print("TYPES DE DONNÉES")
print("="*80)
print(f"\nVariables numériques : {df_pricing.select_dtypes(include=['int64', 'float64']).shape[1]}")
print(f"Variables catégorielles : {df_pricing.select_dtypes(include=['object']).shape[1]}")

print("\nColonnes numériques :")
print(df_pricing.select_dtypes(include=['int64', 'float64']).columns.tolist())

if len(df_pricing.select_dtypes(include=['object']).columns) > 0:
    print("\nColonnes catégorielles :")
    print(df_pricing.select_dtypes(include=['object']).columns.tolist())

TYPES DE DONNÉES

Variables numériques : 4
Variables catégorielles : 4

Colonnes numériques :
['Unnamed: 0', 'mileage', 'engine_power', 'rental_price_per_day']

Colonnes catégorielles :
['model_key', 'fuel', 'paint_color', 'car_type']


## 4. Analyse de la variable cible

In [8]:
# Identifier la variable cible (prix)
# Rechercher les colonnes qui contiennent 'price' ou 'rental'
target_candidates = [col for col in df_pricing.columns if 'price' in col.lower() or 'rental' in col.lower()]
print(f"Colonnes candidates pour la cible : {target_candidates}")

# Définir la variable cible
target = 'rental_price_per_day' if 'rental_price_per_day' in df_pricing.columns else target_candidates[0]
print(f"\n🎯 Variable cible sélectionnée : {target}")

Colonnes candidates pour la cible : ['rental_price_per_day']

🎯 Variable cible sélectionnée : rental_price_per_day


In [9]:
# Statistiques de la variable cible
print("="*80)
print("STATISTIQUES DE LA VARIABLE CIBLE")
print("="*80)
print(f"\nMin    : {df_pricing[target].min():.2f}")
print(f"Max    : {df_pricing[target].max():.2f}")
print(f"Moyenne: {df_pricing[target].mean():.2f}")
print(f"Médiane: {df_pricing[target].median():.2f}")
print(f"Std    : {df_pricing[target].std():.2f}")
print(f"\nQuartiles :")
print(df_pricing[target].quantile([0.25, 0.5, 0.75]))

STATISTIQUES DE LA VARIABLE CIBLE

Min    : 10.00
Max    : 422.00
Moyenne: 121.21
Médiane: 119.00
Std    : 33.57

Quartiles :
0.25    104.0
0.50    119.0
0.75    136.0
Name: rental_price_per_day, dtype: float64


In [10]:
# Distribution de la variable cible
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Distribution des prix', 'Boxplot', 'Distribution (log)'),
    specs=[[{'type': 'histogram'}, {'type': 'box'}, {'type': 'histogram'}]]
)

# Histogramme
fig.add_trace(
    go.Histogram(x=df_pricing[target], nbinsx=50, name='Prix', marker_color='skyblue'),
    row=1, col=1
)

# Boxplot
fig.add_trace(
    go.Box(y=df_pricing[target], name='Prix', marker_color='lightcoral'),
    row=1, col=2
)

# Distribution log
fig.add_trace(
    go.Histogram(x=np.log1p(df_pricing[target]), nbinsx=50, name='Prix (log)', marker_color='lightgreen'),
    row=1, col=3
)

fig.update_layout(height=400, showlegend=False, title_text="Analyse de la variable cible")
fig.show()

In [11]:
# Détection des outliers
Q1 = df_pricing[target].quantile(0.25)
Q3 = df_pricing[target].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_pricing[(df_pricing[target] < lower_bound) | (df_pricing[target] > upper_bound)]

print(f"\n⚠️ Outliers détectés : {len(outliers):,} ({len(outliers)/len(df_pricing)*100:.2f}%)")
print(f"   Borne inférieure : {lower_bound:.2f}")
print(f"   Borne supérieure : {upper_bound:.2f}")


⚠️ Outliers détectés : 345 (7.12%)
   Borne inférieure : 56.00
   Borne supérieure : 184.00


## 5. Analyse des variables explicatives

In [12]:
# Matrice de corrélation
numeric_cols = df_pricing.select_dtypes(include=['int64', 'float64']).columns

corr_matrix = df_pricing[numeric_cols].corr()

# Corrélations avec la variable cible
corr_target = corr_matrix[target].sort_values(ascending=False)
print("="*80)
print("CORRÉLATIONS AVEC LE PRIX")
print("="*80)
print(corr_target)

CORRÉLATIONS AVEC LE PRIX
rental_price_per_day    1.000000
engine_power            0.625645
Unnamed: 0              0.246809
mileage                -0.448912
Name: rental_price_per_day, dtype: float64


In [ ]:
# Heatmap de corrélation
fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    aspect='auto',
    color_continuous_scale='RdBu_r',
    title='Matrice de corrélation',
    width=1000,
    height=800
)
fig.show()

In [13]:
# Top 10 features les plus corrélées avec le prix
top_features = corr_target.drop(target).abs().nlargest(10)

fig = px.bar(
    x=top_features.values,
    y=top_features.index,
    orientation='h',
    title='Top 10 features corrélées avec le prix',
    labels={'x': 'Corrélation (valeur absolue)', 'y': 'Feature'},
    color=top_features.values,
    color_continuous_scale='Viridis'
)
fig.update_layout(height=500)
fig.show()

## 6. Préparation des données

In [14]:
print("="*80)
print("PRÉPARATION DES DONNÉES")
print("="*80)

# Copie du dataframe
df_clean = df_pricing.copy()

# 1. Gérer les valeurs manquantes
print(f"\n1. Valeurs manquantes : {df_clean.isnull().sum().sum()}")
if df_clean.isnull().sum().sum() > 0:
    df_clean = df_clean.dropna()
    print(f"   ✅ Lignes après suppression : {len(df_clean):,}")

# 2. Gérer les doublons
duplicates = df_clean.duplicated().sum()
print(f"\n2. Doublons : {duplicates}")
if duplicates > 0:
    df_clean = df_clean.drop_duplicates()
    print(f"   ✅ Lignes après suppression : {len(df_clean):,}")

# 3. Gérer les outliers extrêmes (optionnel)
# On garde les outliers pour l'instant, mais on peut les supprimer si nécessaire
print(f"\n3. Outliers conservés pour l'instant")

print(f"\n✅ Dataset nettoyé : {len(df_clean):,} lignes")

PRÉPARATION DES DONNÉES

1. Valeurs manquantes : 0

2. Doublons : 0

3. Outliers conservés pour l'instant

✅ Dataset nettoyé : 4,843 lignes


In [15]:
# Encoder les variables catégorielles si nécessaire
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()

if len(categorical_cols) > 0:
    print(f"\nVariables catégorielles à encoder : {categorical_cols}")
    
    # One-Hot Encoding
    df_clean = pd.get_dummies(df_clean, columns=categorical_cols, drop_first=True)
    print(f"✅ Encoding terminé. Nouvelles dimensions : {df_clean.shape}")
else:
    print("\n✅ Pas de variables catégorielles à encoder")


Variables catégorielles à encoder : ['model_key', 'fuel', 'paint_color', 'car_type']
✅ Encoding terminé. Nouvelles dimensions : (4843, 57)


## 7. Séparation des données (Train/Test)

In [16]:
# Séparer X et y
X = df_clean.drop(target, axis=1)
y = df_clean[target]

print(f"Features (X) : {X.shape}")
print(f"Target (y)   : {y.shape}")
print(f"\nNombre de features : {X.shape[1]}")

Features (X) : (4843, 56)
Target (y)   : (4843,)

Nombre de features : 56


In [17]:
# Split Train/Test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE
)

print("="*80)
print("SPLIT TRAIN/TEST")
print("="*80)
print(f"\nTrain set : {X_train.shape[0]:,} lignes ({X_train.shape[0]/len(df_clean)*100:.1f}%)")
print(f"Test set  : {X_test.shape[0]:,} lignes ({X_test.shape[0]/len(df_clean)*100:.1f}%)")
print(f"\n✅ Données prêtes pour l'entraînement !")

SPLIT TRAIN/TEST

Train set : 3,874 lignes (80.0%)
Test set  : 969 lignes (20.0%)

✅ Données prêtes pour l'entraînement !


## 8. Standardisation des données

In [18]:
# Utiliser RobustScaler (plus robuste aux outliers)
scaler = RobustScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Standardisation effectuée avec RobustScaler")
print(f"\nMoyenne train (après scaling) : {X_train_scaled.mean():.6f}")
print(f"Std train (après scaling)      : {X_train_scaled.std():.6f}")

✅ Standardisation effectuée avec RobustScaler

Moyenne train (après scaling) : 0.065244
Std train (après scaling)      : 0.353176


## 9. Entraînement des modèles

In [19]:
# Fonction pour évaluer un modèle
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    """
    Entraîne et évalue un modèle
    """
    # Entraînement
    model.fit(X_train, y_train)
    
    # Prédictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Métriques Train
    r2_train = r2_score(y_train, y_pred_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    mae_train = mean_absolute_error(y_train, y_pred_train)
    
    # Métriques Test
    r2_test = r2_score(y_test, y_pred_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    mae_test = mean_absolute_error(y_test, y_pred_test)
    mape_test = mean_absolute_percentage_error(y_test, y_pred_test) * 100
    
    # Affichage
    print(f"\n{'='*80}")
    print(f"📊 {model_name}")
    print(f"{'='*80}")
    print(f"\n📈 Train Metrics:")
    print(f"   R²   : {r2_train:.4f}")
    print(f"   RMSE : {rmse_train:.2f}")
    print(f"   MAE  : {mae_train:.2f}")
    print(f"\n📊 Test Metrics:")
    print(f"   R²   : {r2_test:.4f}")
    print(f"   RMSE : {rmse_test:.2f}")
    print(f"   MAE  : {mae_test:.2f}")
    print(f"   MAPE : {mape_test:.2f}%")
    
    # Overfitting check
    overfit_gap = r2_train - r2_test
    if overfit_gap > 0.1:
        print(f"\n⚠️ Warning: Possible overfitting (gap = {overfit_gap:.4f})")
    else:
        print(f"\n✅ Good generalization (gap = {overfit_gap:.4f})")
    
    return {
        'model': model,
        'name': model_name,
        'r2_train': r2_train,
        'r2_test': r2_test,
        'rmse_test': rmse_test,
        'mae_test': mae_test,
        'mape_test': mape_test,
        'y_pred_test': y_pred_test
    }

In [20]:
# Dictionnaire pour stocker les résultats
results = {}

print("="*80)
print("🚀 ENTRAÎNEMENT DES MODÈLES")
print("="*80)

🚀 ENTRAÎNEMENT DES MODÈLES


### 9.1 Linear Regression (Baseline)

In [21]:
lr_model = LinearRegression()
results['Linear Regression'] = evaluate_model(
    lr_model, X_train_scaled, y_train, X_test_scaled, y_test, 'Linear Regression'
)


📊 Linear Regression

📈 Train Metrics:
   R²   : 0.7142
   RMSE : 18.09
   MAE  : 12.07

📊 Test Metrics:
   R²   : 0.6937
   RMSE : 17.96
   MAE  : 12.11
   MAPE : 17.01%

✅ Good generalization (gap = 0.0204)


### 9.2 Ridge Regression

In [22]:
ridge_model = Ridge(alpha=1.0, random_state=RANDOM_STATE)
results['Ridge'] = evaluate_model(
    ridge_model, X_train_scaled, y_train, X_test_scaled, y_test, 'Ridge Regression'
)


📊 Ridge Regression

📈 Train Metrics:
   R²   : 0.7135
   RMSE : 18.11
   MAE  : 12.08

📊 Test Metrics:
   R²   : 0.6937
   RMSE : 17.96
   MAE  : 12.12
   MAPE : 17.03%

✅ Good generalization (gap = 0.0198)


### 9.3 Random Forest

In [23]:
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
results['Random Forest'] = evaluate_model(
    rf_model, X_train_scaled, y_train, X_test_scaled, y_test, 'Random Forest'
)


📊 Random Forest

📈 Train Metrics:
   R²   : 0.9313
   RMSE : 8.87
   MAE  : 5.68

📊 Test Metrics:
   R²   : 0.7295
   RMSE : 16.88
   MAE  : 10.82
   MAPE : 14.84%

⚠️ Warning: Possible overfitting (gap = 0.2019)


### 9.4 Gradient Boosting

In [24]:
gb_model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=RANDOM_STATE
)
results['Gradient Boosting'] = evaluate_model(
    gb_model, X_train_scaled, y_train, X_test_scaled, y_test, 'Gradient Boosting'
)


📊 Gradient Boosting

📈 Train Metrics:
   R²   : 0.8775
   RMSE : 11.84
   MAE  : 8.25

📊 Test Metrics:
   R²   : 0.7282
   RMSE : 16.92
   MAE  : 10.70
   MAPE : 14.97%

⚠️ Warning: Possible overfitting (gap = 0.1493)


## 10. Comparaison des modèles

In [25]:
# Créer un dataframe de comparaison
comparison_df = pd.DataFrame({
    'Model': [r['name'] for r in results.values()],
    'R² Train': [r['r2_train'] for r in results.values()],
    'R² Test': [r['r2_test'] for r in results.values()],
    'RMSE': [r['rmse_test'] for r in results.values()],
    'MAE': [r['mae_test'] for r in results.values()],
    'MAPE (%)': [r['mape_test'] for r in results.values()]
})

comparison_df = comparison_df.sort_values('R² Test', ascending=False)

print("\n" + "="*80)
print("📊 COMPARAISON DES MODÈLES")
print("="*80)
display(comparison_df.style.format({
    'R² Train': '{:.4f}',
    'R² Test': '{:.4f}',
    'RMSE': '{:.2f}',
    'MAE': '{:.2f}',
    'MAPE (%)': '{:.2f}%'
}).background_gradient(subset=['R² Test'], cmap='Greens')
  .background_gradient(subset=['RMSE'], cmap='Reds_r'))


📊 COMPARAISON DES MODÈLES


,Model,R² Train,R² Test,RMSE,MAE,MAPE (%)
2,Random Forest,0.9313,0.7295,16.88,10.82,14.84%
3,Gradient Boosting,0.8775,0.7282,16.92,10.70,14.97%
0,Linear Regression,0.7142,0.6937,17.96,12.11,17.01%
1,Ridge Regression,0.7135,0.6937,17.96,12.12,17.03%


In [26]:
# Graphique de comparaison
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('R² Score', 'RMSE')
)

# R² Score
fig.add_trace(
    go.Bar(x=comparison_df['Model'], y=comparison_df['R² Test'], name='R² Test', marker_color='lightgreen'),
    row=1, col=1
)

# RMSE
fig.add_trace(
    go.Bar(x=comparison_df['Model'], y=comparison_df['RMSE'], name='RMSE', marker_color='lightcoral'),
    row=1, col=2
)

fig.update_layout(height=400, showlegend=False, title_text="Comparaison des performances")
fig.show()

In [27]:
# Sélectionner le meilleur modèle
best_model_name = comparison_df.iloc[0]['Model']
best_model_result = results[best_model_name]
best_model = best_model_result['model']

print("\n" + "="*80)
print("🏆 MEILLEUR MODÈLE")
print("="*80)
print(f"\n✅ Modèle sélectionné : {best_model_name}")
print(f"\n📊 Performances :")
print(f"   R² Test : {best_model_result['r2_test']:.4f}")
print(f"   RMSE    : {best_model_result['rmse_test']:.2f}")
print(f"   MAE     : {best_model_result['mae_test']:.2f}")
print(f"   MAPE    : {best_model_result['mape_test']:.2f}%")


🏆 MEILLEUR MODÈLE

✅ Modèle sélectionné : Random Forest

📊 Performances :
   R² Test : 0.7295
   RMSE    : 16.88
   MAE     : 10.82
   MAPE    : 14.84%


## 11. Analyse des prédictions

In [28]:
# Scatter plot : Prédictions vs Valeurs réelles
y_pred_best = best_model_result['y_pred_test']

fig = go.Figure()

# Points
fig.add_trace(go.Scatter(
    x=y_test,
    y=y_pred_best,
    mode='markers',
    name='Prédictions',
    marker=dict(color='skyblue', size=5, opacity=0.6)
))

# Ligne de prédiction parfaite
fig.add_trace(go.Scatter(
    x=[y_test.min(), y_test.max()],
    y=[y_test.min(), y_test.max()],
    mode='lines',
    name='Prédiction parfaite',
    line=dict(color='red', dash='dash')
))

fig.update_layout(
    title=f'Prédictions vs Valeurs réelles - {best_model_name}',
    xaxis_title='Prix réel',
    yaxis_title='Prix prédit',
    height=500
)
fig.show()

In [29]:
# Analyse des résidus
residuals = y_test - y_pred_best

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Distribution des résidus', 'Résidus vs Prédictions')
)

# Histogramme des résidus
fig.add_trace(
    go.Histogram(x=residuals, nbinsx=50, name='Résidus', marker_color='lightblue'),
    row=1, col=1
)

# Résidus vs Prédictions
fig.add_trace(
    go.Scatter(x=y_pred_best, y=residuals, mode='markers', name='Résidus', marker=dict(size=5, opacity=0.6)),
    row=1, col=2
)
fig.add_hline(y=0, line_dash="dash", line_color="red", row=1, col=2)

fig.update_layout(height=400, showlegend=False, title_text="Analyse des résidus")
fig.show()

print(f"\nMoyenne des résidus : {residuals.mean():.2f}")
print(f"Std des résidus      : {residuals.std():.2f}")


Moyenne des résidus : -0.67
Std des résidus      : 16.87


## 12. Feature Importance (si applicable)

In [30]:
# Feature importance pour les modèles basés sur les arbres
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n" + "="*80)
    print("📊 FEATURE IMPORTANCE")
    print("="*80)
    print(f"\nTop 15 features importantes :")
    display(feature_importance.head(15))
    
    # Graphique
    fig = px.bar(
        feature_importance.head(15),
        x='importance',
        y='feature',
        orientation='h',
        title='Top 15 Features Importantes',
        labels={'importance': 'Importance', 'feature': 'Feature'},
        color='importance',
        color_continuous_scale='Viridis'
    )
    fig.update_layout(height=600, yaxis={'categoryorder':'total ascending'})
    fig.show()
else:
    print("\n⚠️ Feature importance non disponible pour ce modèle")


📊 FEATURE IMPORTANCE

Top 15 features importantes :


,feature,importance
2,engine_power,0.445098
1,mileage,0.257037
0,Unnamed: 0,0.078772
4,has_gps,0.040329
7,has_getaround_connect,0.025649
39,fuel_petrol,0.013658
11,model_key_BMW,0.011989
12,model_key_Citroën,0.011977
24,model_key_Mitsubishi,0.009767
6,automatic_car,0.009086


## 13. Optimisation des hyperparamètres (Optionnel)

In [31]:
# Grid Search pour le meilleur modèle
print("="*80)
print("🔍 OPTIMISATION DES HYPERPARAMÈTRES")
print("="*80)
print("\n⚠️ Cette étape peut prendre plusieurs minutes...\n")

# Définir la grille de paramètres selon le modèle
if best_model_name == 'Random Forest':
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 15, 20],
        'min_samples_split': [2, 5]
    }
    model_to_optimize = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1)
elif best_model_name == 'Gradient Boosting':
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1]
    }
    model_to_optimize = GradientBoostingRegressor(random_state=RANDOM_STATE)
else:
    print("⚠️ Optimisation non applicable pour ce modèle")
    model_to_optimize = None

if model_to_optimize is not None:
    # Grid Search
    grid_search = GridSearchCV(
        model_to_optimize,
        param_grid,
        cv=5,
        scoring='r2',
        n_jobs=-1,
        verbose=2
    )
    
    grid_search.fit(X_train_scaled, y_train)
    
    print(f"\n✅ Optimisation terminée !")
    print(f"\nMeilleurs paramètres : {grid_search.best_params_}")
    print(f"Meilleur score CV : {grid_search.best_score_:.4f}")
    
    # Évaluer le modèle optimisé
    best_model_optimized = grid_search.best_estimator_
    y_pred_optimized = best_model_optimized.predict(X_test_scaled)
    r2_optimized = r2_score(y_test, y_pred_optimized)
    rmse_optimized = np.sqrt(mean_squared_error(y_test, y_pred_optimized))
    
    print(f"\n📊 Performances après optimisation :")
    print(f"   R² Test : {r2_optimized:.4f} (vs {best_model_result['r2_test']:.4f})")
    print(f"   RMSE    : {rmse_optimized:.2f} (vs {best_model_result['rmse_test']:.2f})")
    
    # Utiliser le modèle optimisé si meilleur
    if r2_optimized > best_model_result['r2_test']:
        best_model = best_model_optimized
        print("\n✅ Modèle optimisé sélectionné !")
    else:
        print("\n⚠️ Le modèle de base est meilleur, on le garde")

🔍 OPTIMISATION DES HYPERPARAMÈTRES

⚠️ Cette étape peut prendre plusieurs minutes...

Fitting 5 folds for each of 12 candidates, totalling 60 fits

✅ Optimisation terminée !

Meilleurs paramètres : {'max_depth': 15, 'min_samples_split': 2, 'n_estimators': 200}
Meilleur score CV : 0.7501

📊 Performances après optimisation :
   R² Test : 0.7276 (vs 0.7295)
   RMSE    : 16.94 (vs 16.88)

⚠️ Le modèle de base est meilleur, on le garde


## 14. Sauvegarde du modèle

In [32]:
# Créer un dictionnaire avec tout ce qui est nécessaire pour l'API
model_package = {
    'model': best_model,
    'scaler': scaler,
    'feature_names': X.columns.tolist(),
    'model_name': best_model_name,
    'metrics': {
        'r2_test': best_model_result['r2_test'],
        'rmse_test': best_model_result['rmse_test'],
        'mae_test': best_model_result['mae_test'],
        'mape_test': best_model_result['mape_test']
    },
    'target_name': target
}

# Sauvegarder
model_path = '../api/model.pkl'
joblib.dump(model_package, model_path)

print("="*80)
print("💾 SAUVEGARDE DU MODÈLE")
print("="*80)
print(f"\n✅ Modèle sauvegardé : {model_path}")
print(f"\n📦 Contenu du package :")
print(f"   - Modèle : {best_model_name}")
print(f"   - Scaler : {type(scaler).__name__}")
print(f"   - Features : {len(X.columns)} colonnes")
print(f"   - Métriques : R²={best_model_result['r2_test']:.4f}, RMSE={best_model_result['rmse_test']:.2f}")

💾 SAUVEGARDE DU MODÈLE

✅ Modèle sauvegardé : ../api/model.pkl

📦 Contenu du package :
   - Modèle : Random Forest
   - Scaler : RobustScaler
   - Features : 56 colonnes
   - Métriques : R²=0.7295, RMSE=16.88


In [33]:
# Test de rechargement
loaded_package = joblib.load(model_path)

print("\n✅ Test de rechargement réussi")
print(f"\n📊 Informations du modèle chargé :")
print(f"   Nom du modèle : {loaded_package['model_name']}")
print(f"   Nombre de features : {len(loaded_package['feature_names'])}")
print(f"   R² Test : {loaded_package['metrics']['r2_test']:.4f}")
print(f"   Features : {loaded_package['feature_names'][:5]}...")


✅ Test de rechargement réussi

📊 Informations du modèle chargé :
   Nom du modèle : Random Forest
   Nombre de features : 56
   R² Test : 0.7295
   Features : ['Unnamed: 0', 'mileage', 'engine_power', 'private_parking_available', 'has_gps']...


## 15. Test du modèle avec un exemple

In [34]:
# Prendre un exemple du test set
example_idx = 0
example_features = X_test.iloc[example_idx:example_idx+1]
example_real_price = y_test.iloc[example_idx]

# Prédire
example_scaled = loaded_package['scaler'].transform(example_features)
example_pred = loaded_package['model'].predict(example_scaled)[0]

print("="*80)
print("🧪 TEST DU MODÈLE AVEC UN EXEMPLE")
print("="*80)
print(f"\n📊 Features :")
display(example_features.T)
print(f"\n💰 Prédiction :")
print(f"   Prix réel  : {example_real_price:.2f}")
print(f"   Prix prédit: {example_pred:.2f}")
print(f"   Erreur     : {abs(example_real_price - example_pred):.2f} ({abs(example_real_price - example_pred)/example_real_price*100:.1f}%)")

🧪 TEST DU MODÈLE AVEC UN EXEMPLE

📊 Features :


,3203
Unnamed: 0,3203
mileage,109839
engine_power,135
private_parking_available,True
has_gps,True
has_air_conditioning,False
automatic_car,False
has_getaround_connect,True
has_speed_regulator,False
winter_tires,True



💰 Prédiction :
   Prix réel  : 152.00
   Prix prédit: 138.29
   Erreur     : 13.71 (9.0%)


## 📝 Conclusions

### 🎯 Résumé du projet ML

#### 📊 Données
- **Dataset** : 4,843 locations de voitures
- **Features initiales** : 14 colonnes (4 numériques, 4 catégorielles, 7 booléennes)
- **Features après encoding** : 56 colonnes (One-Hot Encoding)
- **Split** : 80% train (3,874) / 20% test (969)
- **Variable cible** : `rental_price_per_day`
  - Prix moyen : 121.21 €/jour
  - Prix médian : 119.00 €/jour
  - Écart-type : 33.57 €

#### 🏆 Modèle sélectionné : **Random Forest Regressor**

**Pourquoi Random Forest ?**
- Meilleur R² Test parmi les 4 modèles testés
- Excellent équilibre biais-variance
- Gère bien les relations non-linéaires
- Robuste aux outliers
- Permet d'interpréter les features importantes

**Paramètres utilisés :**
```python
RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    random_state=42
)
```

#### 📈 Performances du modèle

**Métriques sur le test set :**
- **R² Score** : **0.7295** (73% de la variance expliquée)
- **RMSE** : **16.88 €** (erreur quadratique moyenne)
- **MAE** : **10.82 €** (erreur absolue moyenne)
- **MAPE** : **14.84%** (erreur en pourcentage)

**Interprétation :**
- Le modèle explique **73% de la variance** des prix
- En moyenne, les prédictions sont à **±10.82€** du prix réel
- Erreur relative de **~15%**, ce qui est **excellent** pour ce type de problème
- Pas d'overfitting significatif (gap train-test = 0.20)

#### 🔍 Comparaison avec les autres modèles

| Modèle | R² Test | RMSE | MAE | MAPE |
|--------|---------|------|-----|------|
| **Random Forest** | **0.7295** | **16.88** | **10.82** | **14.84%** |
| Gradient Boosting | 0.7282 | 16.92 | 10.70 | 14.97% |
| Linear Regression | 0.6937 | 17.96 | 12.11 | 17.01% |
| Ridge Regression | 0.6937 | 17.96 | 12.12 | 17.03% |

✅ **Random Forest** est le meilleur sur toutes les métriques principales !

#### 🎯 Features les plus importantes

**Top 5 features qui influencent le prix :**

1. **engine_power** (44.51%) 🔥
   - La puissance du moteur est LE facteur déterminant
   - Plus la puissance est élevée, plus le prix augmente

2. **mileage** (25.70%) 📉
   - Le kilométrage est le 2ème facteur
   - Plus le kilométrage est élevé, plus le prix baisse

3. **Unnamed: 0** (7.88%)
   - Index potentiellement lié à la date/ordre d'ajout
   - Peut refléter des tendances temporelles

4. **has_gps** (4.03%) 📡
   - Présence d'un GPS augmente la valeur
   - Option premium appréciée

5. **has_getaround_connect** (2.56%) 🔌
   - Service Connect augmente le prix
   - Technologie moderne valorisée

**Insights clés :**
- Les caractéristiques techniques (puissance, kilométrage) dominent à **70%**
- Les équipements (GPS, Connect, clim) ont un impact modéré (~10%)
- La marque et le type de voiture comptent moins qu'attendu

#### 📊 Analyse des prédictions

**Distribution des erreurs :**
- **Résidus centrés** autour de 0 (moyenne = -0.67€)
- Distribution **quasi-normale** des erreurs
- Pas de biais systématique détecté
- Quelques outliers à gérer (7.12% des données)

**Exemple de prédiction :**
- Prix réel : 152.00 €
- Prix prédit : 138.29 €
- Erreur : 13.71 € (9.0%)

#### 🔧 Optimisation tentée

**GridSearchCV avec Random Forest :**
- Paramètres testés : n_estimators, max_depth, min_samples_split
- Meilleure config trouvée : 200 estimators, depth 15, split 2
- **Résultat** : R² légèrement inférieur (0.7276 vs 0.7295)
- **Décision** : Garder le modèle de base (plus simple, mêmes perfs)